In [12]:
#!/usr/bin/python2

## UR5/UR10 Inverse Kinematics - Ryan Keating Johns Hopkins University


# ***** lib
import numpy as np
import time
from numpy import linalg
from robot_transformation import numpy_imple


import cmath
import math
from math import cos as cos
from math import sin as sin
from math import atan2 as atan2
from math import acos as acos
from math import asin as asin
from math import sqrt as sqrt
from math import pi as pi

global mat
mat=np.matrix

#nash modified parameters
global d1, a2, a3, a7, d4, d5, d6


UR5NotAuboi5 =True

global d, a, alph
#UR5 D-H Model parameters
if UR5NotAuboi5:
    d = [0.0892, 0, 0, 0.10915, 0.09465,0.0823] #ur5
    a =[0,0,-0.42500,-0.39225,0,0] #ur5
    alpha = [0,math.pi/2, 0, 0, math.pi/2, -math.pi/2]  #ur5
    a2 = -0.425
    a3 = -0.39225
    d4 =  0.10915
    d6 =  0.0819
else:
#Aubo_i5 D-H Model parameters
    d = [0.0985, 0, 0, 0.12150, 0.10250,0.094] 
    a =[0,0,-0.40800,-0.376,0,0] 
    alpha = [0,math.pi/2, 0, 0, math.pi/2, -math.pi/2] 
    a2 = -0.40800
    a3 = -0.376
    d4 =  0.12150
    d6 =  0.094
    auboi5_offset = np.deg2rad([0,90,0,90,0,-180])
#************************************************** FORWARD KINEMATICS
def forward_kinematics(theta): 
    print('theta value',np.rad2deg(theta))
    if False == UR5NotAuboi5:
        theta[2] = -theta[2]
        theta = theta - auboi5_offset
    
    
    homo = np.identity(4)
    homo_2 = np.identity(4)
    homo_3 = np.identity(4)
    
    for i in range(6):
        adjacent_homo =np.array([[cos(theta[i]),-sin(theta[i]),0,a[i]],
                         [sin(theta[i])*cos(alpha[i]), cos(theta[i])*cos(alpha[i]),-sin(alpha[i]),-sin(alpha[i])*d[i]],
                         [sin(theta[i])*sin(alpha[i]),cos(theta[i])*sin(alpha[i]), cos(alpha[i]), cos(alpha[i])*d[i]],
                         [0,0,0,1]])
        homo = np.dot(homo,adjacent_homo)
    return homo
def rpy2rotate(rpy):
    rx,ry,rz =np.deg2rad(np.array(rpy))
    matx = [[1,0,0],[0,np.cos(rx),np.sin(rx)],[0,-np.sin(rx), np.cos(rx)]]
    maty = [[np.cos(ry), 0, -np.sin(ry)],[0,1,0],[np.sin(ry),0,np.cos(ry)]]
    matz = [[np.cos(rz), np.sin(rz),0],[-np.sin(rz), np.cos(rz),0],[0,0,1]]
    tranmartix = np.dot(matx,np.dot(maty,matz))#这里需要确认欧拉角的旋转顺序是Z,Y,X轴
    return tranmartix

def AH(n,th,c  ):#高坐标系往地坐标系转换
    i = n-1
    theta= th[n-1,c]   
    A_i = np.array([[cos(theta),-sin(theta),0,a[i]],
                         [sin(theta)*cos(alpha[i]), cos(theta)*cos(alpha[i]),-sin(alpha[i]),-sin(alpha[i])*d[i]],
                         [sin(theta)*sin(alpha[i]),cos(theta)*sin(alpha[i]), cos(alpha[i]), cos(alpha[i])*d[i]],
                         [0,0,0,1]])
    return A_i
# ************************************************** INVERSE KINEMATICS 

def invKine(desired_pos):# T60
    print('desired_pos\n',desired_pos)
    th = mat(np.zeros((6, 8)))
    P_05 = (desired_pos * mat([0,0, -d6, 1]).T-mat([0,0,0,1 ]).T)
    
    # **** theta1  ****
    
    psi = atan2(P_05[2-1,0], P_05[1-1,0])
    phi = acos(d4 /sqrt(P_05[2-1,0]*P_05[2-1,0] + P_05[1-1,0]*P_05[1-1,0]))
    #The two solutions for theta1 correspond to the shoulder
    #being either left or right
    th[0, 0:4] = pi/2 + psi + phi
    th[0, 4:8] = pi/2 + psi - phi
    th = th.real 
    # **** theta5  ****

    cl = [0, 4]# wrist up or down
    for i in range(0,len(cl)):
            c = cl[i]
            T_10 = linalg.inv(AH(1,th,c))
            T_16 = np.dot(T_10, desired_pos)
            th[4, c:c+2] = + acos((-T_16[1,3]-d4)/d6);
            th[4, c+2:c+4] = - acos((-T_16[1,3]-d4)/d6);

    th = th.real
  # **** theta6 ****
  # theta6 is not well-defined when sin(theta5) = 0 or when T16(1,3), T16(2,3) = 0.

    cl = [0, 2, 4, 6]
    for i in range(0,len(cl)):
        c = cl[i]
        T_10 = linalg.inv(AH(1,th,c))
        T_16 =  np.dot(T_10 , desired_pos) 
        th[5, c:c+2] = atan2((T_16[1,1]/sin(th[4, c])),(-T_16[1,0]/sin(th[4, c])))
      
    th = th.real

    # **** theta3 ****
    cl = [0, 2, 4, 6]
    for i in range(0,len(cl)):
        c = cl[i]
        T_10 = np.linalg.inv(AH(1,th,c))
        T_65 = np.linalg.inv(AH( 6,th,c))
        T_54 = np.linalg.inv(AH( 5,th,c))
        T_14 = np.dot(np.dot(T_10 ,desired_pos) , np.dot(T_65,T_54))
        temp_len = T_14[0,3]**2 + T_14[2,3]**2 
        t3 = cmath.acos((temp_len - a2**2 - a3**2 )/(2 * a2 * a3)) #
        th[2, c] = t3.real
        th[2, c+1] = -t3.real
    
  # **** theta2 and theta 4 ****

    cl = [0, 1, 2, 3, 4, 5, 6, 7]
    for i in range(0,len(cl)):
        c = cl[i]
        T_10 = np.linalg.inv(AH(1,th,c))
        T_65 = np.linalg.inv(AH( 6,th,c))
        T_54 = np.linalg.inv(AH( 5,th,c))
        T_14 = np.dot(np.dot(T_10 ,desired_pos) , np.dot(T_65,T_54))
        temp_len = np.sqrt(T_14[0,3]**2 + T_14[2,3]**2)
        # theta 2
        th[1, c] = atan2(-T_14[2,3], -T_14[0,3]) - asin(-a3* sin(th[2,c])/temp_len)
        # theta 4
        T_32 = linalg.inv(AH( 3,th,c))
        T_21 = linalg.inv(AH( 2,th,c))
        T_34 = np.dot(T_32,np.dot(T_21,T_14))
        th[3, c] = -atan2(T_34[0,1], T_34[0,0])
    th = th.real
    if False == UR5NotAuboi5:
        for iadd in range(6):
            if iadd == 2: 
                th[iadd] = -th[iadd] + auboi5_offset[iadd]
            else:
                th[iadd] = th[iadd] + auboi5_offset[iadd]
    return th


In [15]:
#UR5
## 268.34,54.05,636.98 0.0979,0.8833,-1.0375
#-70.72,-78.52,599.76 0.5576,0.2695,0.8237
#-220.69,45.03,656.97 0.66480,3.6016,-1.3841
#-580.08,256.72,684.64 1.1683,0.4713,-1.5864
#-650.34,-69.47,583.79 0.5164,1.5607,-1.5163
option_index=0
if UR5NotAuboi5:
    th = np.deg2rad(np.array([
        [0,0,0,0,0,0],
        [49.490,-54.125,-81.3975,9.430,-52.3710,-0.147],
        [-87.30,-34.28,-107.62,-152.90,245.70,40.59],
        [-64.59,-31.51,-77.36,-151.92,30.84,40.59],
        [-40.58,-21.63,-47.76,-42.02,28.30,40.60],
        [-10.11,-9.25,-46.46,-109.15,28.31,40.60]
    ]))
    rotate_vectors = np.array([[120.527,-21.274,-8.7098],
          [0.5576,0.2695,0.8237],
          [0.66480,3.6016,-1.3841],
          [1.1683,0.4713,-1.5864],
          [0.5164,1.5607,-1.5163]])
    num_obj = numpy_imple()
    rotate_matrix = num_obj.rot_vec2mat(rotate_vectors[option_index].tolist())
    print('the rotate matrix\n',np.linalg.inv(np.array(rotate_matrix)))
else:
    th = (np.array([
        [0.13007,-0.07936,-0.83157,0.2356,-0.9886,0.0876],
        [22.70,47.36,0,217.35,98.38,-154.29],
        [22.70, 7.3269, -61.594, 15.80, -98.37, 25.71],
        [0,-7.291,-75.69,21.60,-90,0],
        [-15.26,-28.537,27.679,-38.747,65.539,-55.739],
        [11.818,7.10,24.0588,0,0,0],
        [-167.724,52.302,-(35.626),132.345,95.213,19.218],
        [-18.422,-59.115,-(49.768),60.484,6.012,19.924],
        [-40.58,-21.63,-(47.76),-42.02,28.30,40.60],
        [-10.11,-9.25,-(46.46),-109.15,28.31,40.60]
    ]))
    euler_angles = np.array([[143.915,-30.358,-67.19],
                             [90,-31.159,11.813],
                             [-57.160,19.049,-71.90],
                             [72.616,-73.44,1.790]])
    rotate_matrix = rpy2rotate(euler_angles[option_index].tolist())
    print('the rotate matrix\n',np.linalg.inv(np.array(rotate_matrix)))
homo = forward_kinematics(th[option_index])
print('T06 homo\n',homo,'\n')
print('T60 homo\n,',np.linalg.inv(homo),'\n')

inverse_solutions = invKine(homo)
print('initial theta value',np.rad2deg(th[option_index]))
for i in range(6):
    homo = forward_kinematics(np.array(inverse_solutions[:,i]).reshape(6))
    print('T06 homo\n',homo,'\n')
print('inverse solutions\n')
print(np.array(np.rad2deg(inverse_solutions)))

the rotate matrix
 [[ 0.93035084 -0.32541312 -0.16897818]
 [-0.35048415 -0.92463299 -0.14904598]
 [-0.10774129  0.19788923 -0.97428521]]
theta value [ 12.61 -90.     0.   -90.     0.     0.  ]
T06 homo
 [[-9.75878674e-01  1.32878520e-16  2.18313568e-01  4.17961325e-02]
 [-2.18313568e-01 -3.30196335e-17 -9.75878674e-01 -1.86831972e-01]
 [-1.22464680e-16 -1.00000000e+00  6.12323400e-17  1.00110000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]] 

T60 homo
, [[-9.75878674e-01 -2.18313568e-01 -1.22464680e-16  6.09738801e-17]
 [ 1.32878520e-16 -3.30196335e-17 -1.00000000e+00  1.00110000e+00]
 [ 2.18313568e-01 -9.75878674e-01  6.12323400e-17 -1.91450000e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]] 

desired_pos
 [[-9.75878674e-01  1.32878520e-16  2.18313568e-01  4.17961325e-02]
 [-2.18313568e-01 -3.30196335e-17 -9.75878674e-01 -1.86831972e-01]
 [-1.22464680e-16 -1.00000000e+00  6.12323400e-17  1.00110000e+00]
 [ 0.00000000e+00  0.00000000e

In [8]:
homo =np.array( [[1,22,4,4],[3,1,4,5],[0,6,7,8],[8,6,6,6]])
print('inver', np.linalg.inv(homo))
homo


inver [[ 0.01704545  0.15340909 -0.15340909  0.06534091]
 [ 0.05965909  0.03693182 -0.03693182 -0.02130682]
 [-0.30113636 -1.71022727  0.71022727  0.67897727]
 [ 0.21875     1.46875    -0.46875    -0.578125  ]]


array([[ 1, 22,  4,  4],
       [ 3,  1,  4,  5],
       [ 0,  6,  7,  8],
       [ 8,  6,  6,  6]])